<a href="https://colab.research.google.com/github/rmanzi13/Localizador_Recursos_Financieros/blob/main/Localizado_Recursos_Financieros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import pandas as pd

In [143]:
df_contratacionpublica = pd.read_csv('contratacionpublica_spain.csv')

In [144]:
df_contratacionpublica.head(2)

,url,expediente,organo_contratacion,estado,descripcion,presupuesto_base,valor_estimado,tipo_contrato,codigo_cpv,lugar_ejecucion,procedimiento_contratacion,limite_presentacion,adjudicatario,publication_date,pliego_link,pliego_link_tech,pliego_link_admin,last_update,insert_date
0,https://contrataciondelestado.es/wps/poc?uri=d...,2819/2021,Alcaldía del Ayuntamiento de Gibraleón,Resuelta,"Servicio de diseño, impresión y encuadernación...",3300,3300.0,Suministros,{},España - Huelva - Gibraleón,Contrato menor,NaN,ANTONIA SANCHEZ GONZALEZ,9/2/2021 10:05,NaN,NaN,NaN,9/2/2021 10:05,9/2/2021 8:12
1,https://contrataciondelestado.es/wps/poc?uri=d...,2546/2021,Alcaldía del Ayuntamiento de Gibraleón,Resuelta,Contrato Menor de Acometida del nuevo cuarto d...,3777,3777.0,Servicios,{},España - Huelva - Gibraleón,Contrato menor,NaN,"ELECTRICIDAD MARTIN VALEO, S.L.L",9/2/2021 9:20,NaN,NaN,NaN,9/2/2021 9:20,9/2/2021 8:12


In [ ]:
pd.set_option('display.max_rows', None)
unique_organo_contratacion = df_contratacionpublica['organo_contratacion'].value_counts()
print(unique_organo_contratacion)

In [7]:
pd.set_option('display.max_rows', None)
unique_tipo_contrato = df_contratacionpublica['tipo_contrato'].value_counts()
print(unique_tipo_contrato)

Servicios                        2409
Suministros                      1648
Obras                            1428
Privado                           153
Patrimonial                       113
Concesión de Servicios             56
Administrativo especial            25
Concesión de servicios             15
Otros                               5
Gestión de Servicios Públicos       5
Gestion de Servicios Públicos       1
Concesión de obras                  1
Name: tipo_contrato, dtype: int64


In [ ]:
# Creamos un nuevo DataFrame con las variables que utilizaremos
new_df = df_contratacionpublica[['organo_contratacion','tipo_contrato']].copy()

# Displaying the new DataFrame
print(new_df)

In [10]:
new_df.head()

,organo_contratacion,tipo_contrato
0,Alcaldía del Ayuntamiento de Gibraleón,Suministros
1,Alcaldía del Ayuntamiento de Gibraleón,Servicios
2,Alcaldía del Ayuntamiento de Gibraleón,Suministros
3,Subdirección General de la Oficina Presupuesta...,Servicios
4,Alcaldía del Ayuntamiento de Huéscar,Servicios


In [11]:
new_df.shape

(5859, 2)

In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5859 entries, 0 to 5858
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   organo_contratacion  5859 non-null   object
 1   tipo_contrato        5859 non-null   object
dtypes: object(2)
memory usage: 91.7+ KB


In [146]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [147]:
# Creamos una función para procesar los datos
def procesar_datos(new_df):
    # Combinamos las columnas 'organo_contratacion' y 'tipo_contrato' en una nueva columna 'descripcion_contrato'
    new_df['descripcion_contrato'] = new_df['organo_contratacion'] + ' ' + new_df['tipo_contrato']

    # Utilizamos CountVectorizer para convertir la descripción en una matriz de términos
    vectorizer = CountVectorizer()
    matriz_tfidf = vectorizer.fit_transform(new_df['descripcion_contrato'])

    # Calculamos la similitud del coseno entre las descripciones
    similitud_cos = cosine_similarity(matriz_tfidf, matriz_tfidf)

    return similitud_cos

In [148]:
# Obtenemos la matriz de similitud
matriz_similitud = procesar_datos(new_df)

In [150]:
# Función de recomendación

def recomendar_contrato(organismo, tipo_contrato, new_df, matriz_similitud, umbral_similitud=0.5, num_recomendaciones=5):
    try:
        # Encontramos el índice del organismo en el DataFrame
        indice_organismo = new_df[new_df['organo_contratacion'] == organismo].index[0]
    except IndexError:
        print(f"Error: Organismo '{organismo}' no encontrado en el DataFrame.")
        return pd.DataFrame()  # Devolvemos un DataFrame vacío en caso de error

    # Obtenemos la fila correspondiente en la matriz de similitud
    similitudes_organismo = matriz_similitud[indice_organismo]

    # Filtramos las similitudes por umbral
    similitudes_filtradas = [(i, sim) for i, sim in enumerate(similitudes_organismo) if sim > umbral_similitud]

    # Si no hay similitudes suficientes, devolvemos un DataFrame vacío
    if not similitudes_filtradas:
        print("Error: No hay recomendaciones disponibles para los criterios especificados.")
        return pd.DataFrame()

    # Ordenamos las similitudes y tomamos los índices
    indices_ordenados = [i for i, _ in sorted(similitudes_filtradas, key=lambda x: x[1], reverse=True)]

    # Filtramos las recomendaciones por tipo de contrato
    recomendaciones = new_df.loc[indices_ordenados]

    # Verificamos si hay recomendaciones para el tipo de contrato especificado
    recomendaciones_tipo_contrato = recomendaciones[recomendaciones['tipo_contrato'] == tipo_contrato]
    if recomendaciones_tipo_contrato.empty:
        print(f"No hay recomendaciones para el tipo de contrato '{tipo_contrato}' en '{organismo}'.")
        return pd.DataFrame()

    # Limitamos el número de recomendaciones
    recomendaciones_tipo_contrato = recomendaciones_tipo_contrato.head(num_recomendaciones)

    return recomendaciones_tipo_contrato

# Ejemplo de uso
organismo_interes = 'Alcaldía del Ayuntamiento de Gibraleón'
tipo_contrato_interes = 'Suministros'

recomendaciones = recomendar_contrato(organismo_interes, tipo_contrato_interes, new_df, matriz_similitud)
print(recomendaciones)


                       organo_contratacion tipo_contrato  \
0   Alcaldía del Ayuntamiento de Gibraleón   Suministros   
2   Alcaldía del Ayuntamiento de Gibraleón   Suministros   
17  Alcaldía del Ayuntamiento de Gibraleón   Suministros   
18  Alcaldía del Ayuntamiento de Gibraleón   Suministros   
35       Alcaldía del Ayuntamiento de Noja   Suministros   

                                 descripcion_contrato  
0   Alcaldía del Ayuntamiento de Gibraleón Suminis...  
2   Alcaldía del Ayuntamiento de Gibraleón Suminis...  
17  Alcaldía del Ayuntamiento de Gibraleón Suminis...  
18  Alcaldía del Ayuntamiento de Gibraleón Suminis...  
35      Alcaldía del Ayuntamiento de Noja Suministros  
